# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

Using TensorFlow backend.


# Define different types of Keras models

In [2]:
lambda_regularization = 0.20

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(units=1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    #Clear the previous tensorflow graph
    K.clear_session()

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN_6(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        nFeatures = len(selected_features)
        shapeLSTM = (window_size, nFeatures)
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

In [6]:
shape = len(selected_features*window_size)  #Some irrelevant shape

modelRULSN = get_compiled_model(shape, model_type='ann')

tModel = SequenceTunableModelRegression('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', modelRULLSTM, lib_type='keras', data_handler=dHandler_cmaps)
tModel.data_handler.data_scaler = min_max_scaler

# Load Data

In [7]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 125

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616 ..., -0.66666667  0.41085271
   0.42723005]
 [-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.24031008
   0.21817178]
 [-0.31325301 -0.26095487 -0.25894666 ..., -0.33333333  0.36434109
   0.67274234]
 [-0.31325301 -0.48768258 -0.33760972 ..., -0.33333333  0.06976744
   0.2604253 ]
 [-0.30120482 -0.48506649 -0.19074949 ..., -0.33333333  0.03875969
   0.14609224]]
[ 125.  125.  125.  125.  125.]
Testing data (X, y)
[[-0.69879518 -0.24089819 -0.55536799 ..., -0.16666667  0.03875969
   0.27312897]
 [-0.09036145 -0.21037715 -0.37744767 ..., -0.5         0.03875969
   0.01518917]
 [-0.0060241  -0.22127752 -0.10634706 ...,  0.16666667  0.2248062
   0.04

In [8]:
#Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.sequence_length = 30
tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data from memory
Printing shapes

Training data (X, y)
(17731, 30, 14)
(17731,)
Testing data (X, y)
(100, 30, 14)
(100,)
Printing first 5 elements

Training data (X, y)
[[[-0.63253012 -0.18639634 -0.38048616 ..., -0.33333333  0.42635659
    0.44932339]
  [-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.33333333
    0.46202706]
  [-0.31325301 -0.26095487 -0.25894666 ..., -0.66666667  0.25581395
    0.24275062]
  ..., 
  [-0.31325301 -0.48550251 -0.43011479 ..., -0.66666667  0.34883721
    0.07677437]
  [-0.57831325 -0.39873556 -0.36731938 ..., -0.16666667  0.2248062
    0.28555648]
  [-0.40361446 -0.01983867 -0.53308575 ..., -0.66666667  0.41085271
    0.42723005]]

 [[-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.33333333
    0.46202706]
  [-0.31325301 -0.26095487 -0.25894666 ..., -0.66666667  0.25581395
    0.24275062]
  [-0.31325301 -0.48768258 -0.33760972 ..., -0.33333333  0.14728682
    0.32477216]
  ..., 
  [-0.57831325 -0.39873556 -0.36731938 ..., -0.166

# Train the model and test some Tunable Model functionalities

In [8]:
nFeatures = len(selected_features)
shape = nFeatures*window_size
#shape = (window_size, nFeatures)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
modelRULSN = get_compiled_model(shape, model_type='ann')
tModel.epochs = 100
tModel.change_model('ModelRUL_SN_1', modelRULSN, 'keras')
tModel.train_model(learningRate_scheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 1s 49us/step - loss: 7805.9032 - mean_squared_error: 7802.9217
Epoch 2/100
17731/17731 [==============================] - 0s 16us/step - loss: 6169.2668 - mean_squared_error: 6165.4697
Epoch 3/100
17731/17731 [==============================] - 0s 14us/step - loss: 4605.2386 - mean_squared_error: 4599.4911
Epoch 4/100
17731/17731 [==============================] - 0s 12us/step - loss: 3242.9761 - mean_squared_error: 3234.4806
Epoch 5/100
17731/17731 [==============================] - 1s 31us/step - loss: 2198.2085 - mean_squared_error: 2186.6243
Epoch 6/100
17731/17731 [==============================] - 1s 34us/step - loss: 1509.5119 - mean_squared_error: 1494.9559
Epoch 7/100
17731/17731 [==============================] - 1s 31us/step - loss: 1122.7456 - mean_squared_error: 1105.7636
Epoch 8/100
17731/17731 [==============================] - 1s 35us/step - loss: 934.1898 - mean_squared_error: 915.5245
Epoch 9/100
17731/17731 [=

17731/17731 [==============================] - 0s 8us/step - loss: 333.9415 - mean_squared_error: 277.6121
Epoch 68/100
17731/17731 [==============================] - 0s 10us/step - loss: 332.7000 - mean_squared_error: 276.3191
Epoch 69/100
17731/17731 [==============================] - 0s 8us/step - loss: 331.5445 - mean_squared_error: 275.1442
Epoch 70/100
17731/17731 [==============================] - 0s 10us/step - loss: 330.2247 - mean_squared_error: 273.8423 0s - loss: 327.4368 - mean_squared_error: 271. - ETA: 0s - loss: 329.9858 - mean_squared_error: 273.60
Epoch 71/100
17731/17731 [==============================] - 0s 10us/step - loss: 329.0077 - mean_squared_error: 272.6395
Epoch 72/100
17731/17731 [==============================] - 0s 8us/step - loss: 327.9967 - mean_squared_error: 271.6520
Epoch 73/100
17731/17731 [==============================] - 0s 8us/step - loss: 326.7755 - mean_squared_error: 270.4968
Epoch 74/100
17731/17731 [==============================] - 0s 10us

In [9]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 593us/step
scores
Engine 1, Predicted RUL [ 113.57201385], Rounded RUL 113.0, Real RUL 112.0
Engine 2, Predicted RUL [ 109.90829468], Rounded RUL 109.0, Real RUL 98.0
Engine 3, Predicted RUL [ 68.32557678], Rounded RUL 68.0, Real RUL 69.0
Engine 4, Predicted RUL [ 83.54307556], Rounded RUL 83.0, Real RUL 82.0
Engine 5, Predicted RUL [ 95.82693481], Rounded RUL 95.0, Real RUL 91.0
Engine 6, Predicted RUL [ 100.75080872], Rounded RUL 100.0, Real RUL 93.0
Engine 7, Predicted RUL [ 104.21243286], Rounded RUL 104.0, Real RUL 91.0
Engine 8, Predicted RUL [ 85.0142746], Rounded RUL 85.0, Real RUL 95.0
Engine 9, Predicted RUL [ 113.89598083], Rounded RUL 113.0, Real RUL 111.0
Engine 10, Predicted RUL [ 87.87527466], Rounded RUL 87.0, Real RUL 96.0
Engine 11, Predicted RUL [ 92.10449219], Rounded RUL 92.0, Real RUL 97.0
Engine 12, Predicted RUL [ 100.19592285], Rounded RUL 100.0, Real RUL 124.0
Engine 13, Predicted RUL [ 88.80731201], Rounded RUL 88